## 4.3.4 층 정규화
* 책의 코드가 동작하도록 원서의 내용에서 일부 코드를 추가했습니다.

In [1]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):
    """
    Layer Normalization 구현
    
    각 입력 샘플에 대해 마지막 차원을 기준으로 정규화를 수행합니다.
    Transformer 아키텍처에서 핵심적으로 사용되는 정규화 기법입니다.
    
    Args:
        dimension (int): 정규화할 차원의 크기
        gamma (torch.Tensor, optional): 스케일 파라미터 (기본값: 1로 초기화)
        beta (torch.Tensor, optional): 시프트 파라미터 (기본값: 0으로 초기화)
        epsilon (float): 수치적 안정성을 위한 작은 값 (기본값: 1e-5)
    """
    def __init__(self, dimension, gamma=None, beta=None, epsilon=1e-5):
        super(LayerNorm, self).__init__()
        self.epsilon = epsilon
        
        # gamma (scale parameter): 학습 가능한 파라미터로 1로 초기화
        self.gamma = gamma if gamma is not None else nn.Parameter(torch.ones(dimension))
        
        # beta (shift parameter): 학습 가능한 파라미터로 0으로 초기화
        self.beta = beta if beta is not None else nn.Parameter(torch.zeros(dimension))

    def forward(self, x):
        """
        LayerNorm 순전파
        
        Args:
            x (torch.Tensor): 입력 텐서 (batch_size, seq_len, dimension)
            
        Returns:
            torch.Tensor: 정규화된 출력 텐서 (입력과 같은 형태)
        """
        # 마지막 차원에 대해 평균 계산 (keepdim=True로 차원 유지)
        mean = x.mean(-1, keepdim=True)
        
        # 마지막 차원에 대해 분산 계산 (unbiased=False: 모집단 분산)
        variance = x.var(-1, keepdim=True, unbiased=False)
        
        # 정규화: (x - mean) / sqrt(variance + epsilon)
        x_normalized = (x - mean) / torch.sqrt(variance + self.epsilon)
        
        # 스케일링과 시프트: gamma * x_normalized + beta
        return self.gamma * x_normalized + self.beta

# 하이퍼파라미터 설정
embedding_dim = 512  # 임베딩 차원
batch_size = 2       # 배치 크기
seq_len = 10         # 시퀀스 길이

# 더미 입력 데이터 생성 (평균이 0이 아니고 분산이 1이 아닌 데이터)
inputs = torch.randn(batch_size, seq_len, embedding_dim) * 3 + 2  # 평균=2, 표준편차=3
print(f"입력 텐서 형태: {inputs.shape}")
print(f"입력 데이터 통계:")
print(f"  - 전체 평균: {inputs.mean().item():.4f}")
print(f"  - 전체 표준편차: {inputs.std().item():.4f}")
print(f"  - 마지막 차원 평균 (첫 번째 샘플): {inputs[0, 0, :].mean().item():.4f}")
print(f"  - 마지막 차원 표준편차 (첫 번째 샘플): {inputs[0, 0, :].std().item():.4f}")

# LayerNorm 인스턴스 생성
layer_norm = LayerNorm(embedding_dim)

# 모델 파라미터 확인
print(f"\nLayerNorm 파라미터:")
print(f"  - gamma 형태: {layer_norm.gamma.shape}")
print(f"  - beta 형태: {layer_norm.beta.shape}")
print(f"  - epsilon: {layer_norm.epsilon}")

# 순전파 실행
outputs = layer_norm(inputs)

print(f"\n출력 텐서 형태: {outputs.shape}")
print(f"출력 데이터 통계:")
print(f"  - 전체 평균: {outputs.mean().item():.4f}")
print(f"  - 전체 표준편차: {outputs.std().item():.4f}")
print(f"  - 마지막 차원 평균 (첫 번째 샘플): {outputs[0, 0, :].mean().item():.6f}")
print(f"  - 마지막 차원 표준편차 (첫 번째 샘플): {outputs[0, 0, :].std().item():.6f}")

# PyTorch 내장 LayerNorm과 비교
builtin_layer_norm = nn.LayerNorm(embedding_dim)
builtin_outputs = builtin_layer_norm(inputs)

print(f"\nPyTorch 내장 LayerNorm 비교:")
print(f"  - 내장 LayerNorm 평균: {builtin_outputs[0, 0, :].mean().item():.6f}")
print(f"  - 내장 LayerNorm 표준편차: {builtin_outputs[0, 0, :].std().item():.6f}")

# 정규화 효과 확인: 각 샘플의 각 위치에서 평균=0, 표준편차=1인지 확인
print(f"\n정규화 검증 (처음 3개 샘플):")
for i in range(min(3, batch_size)):
    for j in range(min(3, seq_len)):
        sample_mean = outputs[i, j, :].mean().item()
        sample_std = outputs[i, j, :].std().item()
        print(f"  샘플[{i}][{j}] - 평균: {sample_mean:.6f}, 표준편차: {sample_std:.6f}")

# 학습 가능한 파라미터 확인
print(f"\n학습 가능한 파라미터:")
for name, param in layer_norm.named_parameters():
    print(f"  {name}: {param.shape}, requires_grad={param.requires_grad}")

# 간단한 학습 예시
print(f"\n학습 예시:")
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(layer_norm.parameters(), lr=0.01)

# 가상의 target
target = torch.randn_like(outputs)

# 초기 loss
initial_loss = criterion(outputs, target)
print(f"초기 Loss: {initial_loss.item():.4f}")

# 한 번의 학습 스텝
optimizer.zero_grad()
loss = criterion(layer_norm(inputs), target)
loss.backward()
optimizer.step()

print(f"한 스텝 후 Loss: {loss.item():.4f}")
print(f"Gamma 변화: {layer_norm.gamma[:5]}...")  # 처음 5개 값만 출력
print(f"Beta 변화: {layer_norm.beta[:5]}...")    # 처음 5개 값만 출력

입력 텐서 형태: torch.Size([2, 10, 512])
입력 데이터 통계:
  - 전체 평균: 1.9785
  - 전체 표준편차: 3.0473
  - 마지막 차원 평균 (첫 번째 샘플): 2.1421
  - 마지막 차원 표준편차 (첫 번째 샘플): 2.9843

LayerNorm 파라미터:
  - gamma 형태: torch.Size([512])
  - beta 형태: torch.Size([512])
  - epsilon: 1e-05

출력 텐서 형태: torch.Size([2, 10, 512])
출력 데이터 통계:
  - 전체 평균: 0.0000
  - 전체 표준편차: 1.0000
  - 마지막 차원 평균 (첫 번째 샘플): -0.000000
  - 마지막 차원 표준편차 (첫 번째 샘플): 1.000978

PyTorch 내장 LayerNorm 비교:
  - 내장 LayerNorm 평균: 0.000000
  - 내장 LayerNorm 표준편차: 1.000978

정규화 검증 (처음 3개 샘플):
  샘플[0][0] - 평균: -0.000000, 표준편차: 1.000978
  샘플[0][1] - 평균: 0.000000, 표준편차: 1.000977
  샘플[0][2] - 평균: 0.000000, 표준편차: 1.000977
  샘플[1][0] - 평균: 0.000000, 표준편차: 1.000978
  샘플[1][1] - 평균: -0.000000, 표준편차: 1.000978
  샘플[1][2] - 평균: 0.000000, 표준편차: 1.000978

학습 가능한 파라미터:
  gamma: torch.Size([512]), requires_grad=True
  beta: torch.Size([512]), requires_grad=True

학습 예시:
초기 Loss: 2.0129
한 스텝 후 Loss: 2.0129
Gamma 변화: tensor([0.9900, 0.9900, 0.9900, 0.9900, 0.9900], grad_fn=<SliceBackward0